In [1]:
import os
import pandas as pd
import torch as tc
import numpy as np
import matplotlib as plt
import datetime as dttt

pd.set_option('display.max_columns',500)

In [21]:
if os.path.isfile("data\\2023.csv"):
    data = pd.read_csv("data\\2023.csv")

In [ ]:
split_loc = "data\\set"
if not os.path.exists(split_loc):
    os.makedirs(split_loc)

In [18]:
data.dtypes

date             datetime64[ns]
course                   object
off                      object
race_name                object
class                     int64
pattern                  object
age_band                 object
dist_m                  float64
going                    object
surface                  object
ran                       int64
pos                      object
draw                      int64
ovr_btn                  object
horse                    object
age                       int64
sex                      object
lbs                       int64
hg                       object
secs                     object
dec                     float64
jockey                   object
trainer                  object
or                       object
rpr                      object
sire                     object
dam                      object
damsire                  object
course_code               int64
date_time        datetime64[ns]
quarter                   int64
pattern_

### Clean up data

In [22]:
data.drop(columns=["region","comment","num"], inplace=True)
data = data[data.secs != "-"]
data = data[data["or"] != "–"]
data = data[data.rpr != "–"]
data = data[data["pos"].str.isdigit()]
data["pos"] = data["pos"].astype(int)
data["class"] = np.where(~data["pattern"].isna(), "g", data["class"])
name_match = ["Classic Mile", "Classic Cup", "Derby"]
mapCL = {"Class 5": 0, "Class 4":0, "Class 3":1, "Class 2": 2, "Class 1":3, "l":4, "g":5, "NaN":0}
data["class"] = np.where(data["class"].isna() & data["race_name"].str.contains('|'.join(name_match)),"l",data["class"])
data["class"].fillna(value="NaN")
data["class"].replace(mapCL,inplace=True)
mapC = {"Happy Valley (HK)":0, "Sha Tin (HK)":1}
data["course_code"] = data["course"].replace(mapC)
data["date_time"] = pd.to_datetime(data["date"] + " " + data["off"])
data["date"] = pd.to_datetime(data["date"])
data["quarter"] = (data["date"].dt.month+2)//3
mapPT = {"NaN":0,"Group 3":1,"Group 2": 2, "Group 1":3}
data["pattern"].fillna(value="NaN")
data["pattern_code"] = data["pattern"].replace(mapPT)
mapAB = {"2yo+":0,"3yo":1,"3yo+":2,"4yo":3,"4yo+":4}
data["age_band_code"] = data["age_band"].replace(mapAB)
mapDist = {distance: i for i, distance in enumerate(sorted(data["dist_m"].unique()))}
data["dist_code"] = data["dist_m"].replace(mapDist)
mapGO = {surf: i for i, surf in enumerate(data["going"].unique())}
data["going_code"] = data["going"].replace(mapGO)
mapSF = {surf: i for i, surf in enumerate(data["surface"].unique())}
data["surface_code"] = data["surface"].replace(mapSF)
mapSX = {sex: i for i, sex in enumerate(data["sex"].unique())}
data["sex_code"] = data["sex"].replace(mapSX)
data.fillna(value={"hg":"NaN","class":"g"},inplace=True)
data["in_place"] = np.where(data["pos"] <= 3,1,0)
data["decLog"] = np.log1p(data["dec"])

In [25]:
data.dtypes

date             datetime64[ns]
course                   object
off                      object
race_name                object
class                     int64
pattern                  object
age_band                 object
dist_m                  float64
going                    object
surface                  object
ran                       int64
pos                       int32
draw                      int64
ovr_btn                  object
horse                    object
age                       int64
sex                      object
lbs                       int64
hg                       object
secs                     object
dec                     float64
jockey                   object
trainer                  object
or                       object
rpr                      object
sire                     object
dam                      object
damsire                  object
course_code               int64
date_time        datetime64[ns]
quarter                   int64
pattern_